# Corpusanalyse UiTdatabank

We hebben de volgende bibliotheken nodig:

In [1]:
from pandas import read_excel, read_csv, DataFrame
from datetime import datetime
from codecs import open

Dan lezen we lezen de gegevens in, die we inperken tot 2014.

In [2]:
df = read_excel("gegevens.xlsx", sheetname='theaterdans1014')
df = df[df["Datum tot"].between(datetime(2014, 1, 1), datetime(2014, 12, 31))]

We kunnen kort inspecteren hoe deze data eruitzien.

In [3]:
df.head()

,Id,tekst,Gemeente,Datum tot
747,17915,Shen Yun brengt 5.000 jaar Chinese beschaving ...,Brussel,2014-04-02
748,17915,Shen Yun brengt 5.000 jaar Chinese beschaving ...,Brussel,2014-04-03
749,17915,Shen Yun brengt 5.000 jaar Chinese beschaving ...,Brussel,2014-04-04
750,17915,Shen Yun brengt 5.000 jaar Chinese beschaving ...,Brussel,2014-04-05
751,17915,Shen Yun brengt 5.000 jaar Chinese beschaving ...,Brussel,2014-04-05


We zien dat elke lijn een event beschrijving bevat, de plaats waar een event plaatsvindt, en ook de datum. Merk op dat events die op meerdere dagen plaatsvinden een aparte lijn krijgen. We zullen hiermee rekening houden in de interpretatie van de resultaten.

Voor onze analyse hebben we ook nood aan een lijst van namen van landen, coordinaten voor de plaatsnamen, en ook een manueel gemaakte mapping om de plaatsnamen in de UiTdatabank gegevens te normaliseren.

In [4]:
with open("landen.txt", "r", "utf-8") as f:
    landnamen = f.readlines()
    landnamen = [n.strip() for n in landnamen]
typering = read_csv("typeerpostcode.csv", delimiter=';')
coord = read_csv("coordinaten.csv", delimiter=';')

Nu gaan we voor ieder event in de UiTdatabankgegevens na welk land er vermeld wordt in de beschrijving van dat event. We houden ook al onmiddellijk bij wat de genormaliseerde naam is van de gemeente en de coordinaten van het centrum.

In [5]:
data = {}
for i in df.index:
    tekst = df["tekst"][i]
    gemeente = df["Gemeente"][i]
    for landnaam in landnamen:
        if landnaam in tekst:
            typeringlijn = typering[typering["Gemeente Origineel"] == gemeente]
            fusiegemeente = typeringlijn["Fusiegemeente"].values[0]
            provincie = typeringlijn["Province (English)"].values[0]
            lat = coord[coord["Fusiegemeente"] == fusiegemeente]["latitude"].values[0]
            lon = coord[coord["Fusiegemeente"] == fusiegemeente]["longitude"].values[0]

            if fusiegemeente not in data:
                data[fusiegemeente] = {"latitude": float(lat.replace(',', '.')),
                                       "longitude": float(lon.replace(',', '.')),
                                       "provincie": provincie,
                                       "landen": {}
                                       }

            if landnaam not in data[fusiegemeente]["landen"]:
                data[fusiegemeente]["landen"][landnaam] = 0
            data[fusiegemeente]["landen"][landnaam] += 1

We kunnen nu deze gegevens samenballen in een tabel:

In [6]:
lines = []
for fusiegemeente in data:
    for land in data[fusiegemeente]["landen"]:
        lines.append([fusiegemeente,
                      float(data[fusiegemeente]["latitude"]),
                      float(data[fusiegemeente]["longitude"]),
                      land,
                      data[fusiegemeente]["landen"][land]
                     ])
res = DataFrame(lines, columns=['Gemeente', 'Latitude', 'Longitude', 'Vermeld land', 'Frequentie'])

De tabel bevat dus de volgende informatie:

In [7]:
res.head()

,Gemeente,Latitude,Longitude,Vermeld land,Frequentie
0,Brasschaat,51.292270,4.494190,Nederland,3
1,Brasschaat,51.292270,4.494190,Mexico,1
2,Brasschaat,51.292270,4.494190,Kroatië,1
3,Brasschaat,51.292270,4.494190,Frankrijk,1
4,Erpe-Mere,50.914137,3.956833,Nederland,5


Als voorbeeld kan de eerste lijn gelezen worden als "in de gemeente Denderleeuw (coordinaten 50.88, 4.07) wordt India 1 keer vermeld".

We schrijven tot slot deze tabel uit naar een CSV bestand voor verdere analyse.

In [8]:
res.to_csv("landsvermeldingen.csv")